## First, imports:

In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy import units

In [ ]:
# Import my library:

import os
import sys

apt_path = os.path.abspath(os.path.join('..', 'apostletools'))
sys.path.append(apt_path)

import snapshot
import simulation
import dataset_comp

In [ ]:
import importlib
importlib.reload(snapshot)
importlib.reload(dataset_comp)
importlib.reload(simulation)

# Rotation Curves of Dwarf Galaxies

## Motivation

I plot bundles of rotation curves of satellites and isolated galaxies, binning them by mass. I expect that the curvaton rotation curves are "shallower" around the centre. In addition, I try to see if there is difference in the diversity of density profiles between the models, or if the difference is simply a characteristic shift towards shallower profiles, as found in Lovell (2016) for the sterile neutrino model.

---

## Set Parameters for the Plots

Choose the snapshot and the simulations, and define M31 and MW in each simulation. Also, set the colors used for each simulation:

In [ ]:
snap_id = 127
sim_ids = ['V1_MR_fix', 'V1_MR_curvaton_p082_fix']
names = ['plain-LCDM', 'curv-p082']
colors = ['black', 'red']

m31 = [(1,0), (1,0)]
mw = [(2,0), (1,1)]

Set lower limit for the value of $v_\mathrm{max}$ of the galaxies to be included (separately for satellites and isolated galaxies):

In [ ]:
sat_down = 10
isol_down = 10

---

## Retrieve Data

### Create a Dictionary

For easy handling of the relevant data, define a data dictionary that, at the top level, has entries for all simulations. Under each simulation entry, add items for the needed datasets and, under the 'Selections' key, a sub-dictionary of masking arrays for each needed condition (e.g. satellite, luminous, $v_\mathrm{max}$ inside range, etc.).

First, add the above definitions into the data dict:

In [ ]:
data = {}
for name, sim_id, m31_ns, mw_ns, col in zip(names, sim_ids, m31, mw, colors):
    data[name] = {'snapshot': snapshot.Snapshot(sim_id, snap_id, name=name),
                  'M31_identifier': m31_ns,
                  'MW_identifier': mw_ns,
                  'Color': col}

Compute the mass bins for satellites and isolated galaxies. I use four bins of equal width in dex. The range used for binning is from <tt>sat_down</tt> (or <tt>isol_down</tt>) to $v_\mathrm{max}$ of the most massive galaxy

In [ ]:
sat_up = 0
isol_up = 0

for name, sim_data in data.items():
    # Get data:
    snap = sim_data['snapshot']
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    
    # Split into satellites:
    mask_m31, mask_mw, mask_isol = dataset_comp.split_satellites_by_distance(
        snap, sim_data['M31_identifier'], sim_data['MW_identifier'])
    
    # Update upper mass limits:
    sat_up = max(sat_up, np.max(vmax[np.logical_or(mask_m31, mask_mw)]))
    isol_up = max(isol_up, np.max(vmax[mask_isol]))

# To include the most massive galaxy inside the bin:
sat_up += sat_up * 0.001 
isol_up += isol_up * 0.001 

sat_up = max(sat_up, isol_up)
isol_up = sat_up

# Compute satellite bins:
down = np.log10(sat_down); up = np.log10(sat_up)
dx = (up - down) / 4
print(dx)
binc = np.array([(down + dx/2) + dx*n for n in range(4)])
dex_bins = np.array([[c-dx/2,c,c+dx/2] for c in binc])
sat_bins = 10**dex_bins
print(sat_bins)

# Compute isolated galaxy bins:
down = np.log10(isol_down); up = np.log10(isol_up)
dx = (up - down) / 4
print(dx)
binc = np.array([(down + dx/2) + dx*n for n in range(4)])
dex_bins = np.array([[c-dx/2,c,c+dx/2] for c in binc])
isol_bins = 10**dex_bins
print(isol_bins)

Then, loop over simulations, retrieve data, compute masking arrays, and add to the dictionary:

In [ ]:
for name, sim_data in data.items():
    # Get data:
    snap = sim_data['snapshot']
    max_point = snap.get_subhalos("Max_Vcirc", "Extended")
    vmax = max_point[:,0] * units.cm.to(units.km)
    rmax = max_point[:,1] * units.cm.to(units.kpc)
    
    # Retrieve rotation curves and split by subhalo:
    rot_curves = snap.get_subhalos(
        'Vcirc', h5_group='Extended/RotationCurve/All')
    sub_offset = snap.get_subhalos(
        'SubOffset', h5_group='Extended/RotationCurve/All')
    v_circ = rot_curves[:,0] * units.cm.to(units.km)
    radii = rot_curves[:,1] * units.cm.to(units.kpc)
    v_circ = np.array(np.split(v_circ, sub_offset[1:]))
    radii = np.array(np.split(radii, sub_offset[1:]))

    # Split into satellites:
    mask_m31, mask_mw, mask_isol = dataset_comp.split_satellites_by_distance(
        snap, sim_data['M31_identifier'], sim_data['MW_identifier'])
    
    # Compute masks for vmax bins:
    sat_bin_masks = [dataset_comp.prune_vmax(snap, low_lim=b[0], up_lim=b[2])
                     for b in sat_bins]
    isol_bin_masks = [dataset_comp.prune_vmax(snap, low_lim=b[0], up_lim=b[2])
                     for b in isol_bins]

    # Compute other masking arrays:
    mask_lum, mask_dark = dataset_comp.split_luminous(snap)
    mask_shallow = rmax > 10
    
    # Add selections (masking arrays):
    data[name]['Selections'] = {
        'M31': mask_m31,
        'MW': mask_mw,
        'Satellite': np.logical_or(mask_m31, mask_mw),
        'Isolated': mask_isol,
        'VmaxBins': {
            'Satellite': {
                'Low': sat_bin_masks[0], 
                'SemiLow': sat_bin_masks[1],
                'SemiHigh': sat_bin_masks[2],
                'High': sat_bin_masks[3]  
            },
            'Isolated': {
                'Low': isol_bin_masks[0], 
                'SemiLow': isol_bin_masks[1],
                'SemiHigh': isol_bin_masks[2],
                'High': isol_bin_masks[3]  
            }
        },
        'Luminous': mask_lum,
        'Dark': mask_dark,
        'Shallow': mask_shallow
    }
    
    # Add datasets to dictionary:
    data[name]['Vmax'] = vmax
    data[name]['Rmax'] = rmax
    data[name]['Vcirc'] = v_circ
    data[name]['Radius'] = radii

---

In [ ]:
# Line transparency:
a = 0.7

## Plot Unmodified Rotation Curve Bundles

### Plot Satellites

In [ ]:
# Choose font sizes:
parameters = {'font.size': 12,
              'figure.titlesize': 14,
              'axes.labelsize': 12,
              'xtick.labelsize': 10,
              'ytick.labelsize': 10,
              'legend.fontsize': 12}

# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

In [ ]:
# Store the figure into a variable ("plain": unmodified rot. curves), for saving etc.:
fig_sat_plain, axes = plt.subplots(nrows=2, ncols=2, sharex='row', sharey='row', figsize=(6, 6))

plt.subplots_adjust(wspace=0.08)
plt.subplots_adjust(hspace=0.2)

# Set axis limits:
axes[0,0].set_xlim(0,10)
axes[0,0].set_ylim(0,35)
axes[1,0].set_xlim(0,30)
axes[1,0].set_ylim(0,90)

# Set labels:
s=14
axes[1,0].set_xlabel('$r[\mathrm{kpc}]$')
axes[1,1].set_xlabel('$r[\mathrm{kpc}]$')
axes[0,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$')
axes[1,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$')

fig_sat_plain.suptitle('Satellite Galaxies', y=0.95)

Plot dark:

In [ ]:
dark_col = ['gray', 'pink']; b = 0.5

# Plot rotation curves:
for i, (name, entry) in enumerate(data.items()):
    
    # Plot low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['Low'],
                                  entry['Selections']['Dark'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,0].plot(r, v, c=dark_col[i], alpha=b, linestyle='dashed')
    
    # Plot semi-low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['SemiLow'],
                                  entry['Selections']['Dark'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,1].plot(r, v, c=dark_col[i], alpha=b, linestyle='dashed')
        
    # Plot semi-high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['SemiHigh'],
                                  entry['Selections']['Dark'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,0].plot(r, v, c=dark_col[i], alpha=b, linestyle='dashed')
        
    # Plot high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['High'],
                                  entry['Selections']['Dark'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,1].plot(r, v, c=dark_col[i], alpha=b, linestyle='dashed')

fig_sat_plain

Plot luminous:

In [ ]:
# Plot rotation curves:
for i, (name, entry) in enumerate(data.items()):
    
    # Plot low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['Low'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,0].plot(r, v, c=entry['Color'], alpha=a)
    
    # Plot semi-low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['SemiLow'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,1].plot(r, v, c=entry['Color'], alpha=a)
        
    # Plot semi-high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['SemiHigh'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,0].plot(r, v, c=entry['Color'], alpha=a)
        
    # Plot high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['High'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,1].plot(r, v, c=entry['Color'], alpha=a)

fig_sat_plain

In [ ]:
axes[0,0].text(0.08, 0.92,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[0,1], sat_bins[0,2] - sat_bins[0,1], 
                   sat_bins[0,1] - sat_bins[0,0]),
               transform=axes[0,0].transAxes, 
               horizontalalignment='left',
               verticalalignment='top')
axes[0,1].text(0.92, 0.08,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[1,1], sat_bins[1,2] - sat_bins[1,1], 
                   sat_bins[1,1] - sat_bins[1,0]),
               transform=axes[0,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom')
axes[1,0].text(0.08, 0.92,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[2,1], sat_bins[2,2] - sat_bins[2,1], 
                   sat_bins[2,1] - sat_bins[2,0]),
               transform=axes[1,0].transAxes,
               horizontalalignment='left',
               verticalalignment='top')
axes[1,1].text(0.92, 0.08,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[3,1], sat_bins[3,2] - sat_bins[3,1], 
                   sat_bins[3,1] - sat_bins[3,0]),
               transform=axes[1,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom')

fig_sat_plain

### Save the Figure

In [ ]:
filename = 'sat_rotcurves.png'
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig_sat_plain.savefig(filename, dpi=300, bbox_inches='tight')

### Plot Isolated Galaxies

In [ ]:
# Choose font sizes:
parameters = {'font.size': 12,
              'figure.titlesize': 14,
              'axes.labelsize': 12,
              'xtick.labelsize': 10,
              'ytick.labelsize': 10,
              'legend.fontsize': 12}

# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

In [ ]:
# Store the figure into a variable ("plain": unmodified rot. curves), for saving etc.:
fig_isol_plain, axes = plt.subplots(nrows=2, ncols=2, sharex='row', sharey='row', figsize=(6, 6))

plt.subplots_adjust(wspace=0.08)
plt.subplots_adjust(hspace=0.2)

# Set axis limits:
axes[0,0].set_xlim(0,10)
axes[0,0].set_ylim(0,35)
axes[1,0].set_xlim(0,30)
axes[1,0].set_ylim(0,90)

# Set labels:
axes[1,0].set_xlabel('$r[\mathrm{kpc}]$')
axes[1,1].set_xlabel('$r[\mathrm{kpc}]$')
axes[0,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$')
axes[1,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$')

fig_isol_plain.suptitle('Isolated Galaxies', y=0.95)

Plot dark:

In [ ]:
dark_col = ['gray', 'pink']; b = 0.5

# Plot rotation curves:
for i, (name, entry) in enumerate(data.items()):
    
    # Plot low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['Low'],
                                  entry['Selections']['Dark'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask] + 20
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,0].plot(r, v, c=dark_col[i], alpha=b)
    
    # Plot semi-low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['SemiLow'],
                                  entry['Selections']['Dark'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask] + 20
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,1].plot(r, v, c=dark_col[i], alpha=b)
        
    # Plot semi-high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['SemiHigh'],
                                  entry['Selections']['Dark'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,0].plot(r, v, c=dark_col[i], alpha=b)
        
    # Plot high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['High'],
                                  entry['Selections']['Dark'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,1].plot(r, v, c=dark_col[i], alpha=b)

fig_isol_plain

Plot luminous:

In [ ]:
# Plot rotation curves:
for i, (name, entry) in enumerate(data.items()):
    
    # Plot low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['Low'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,0].plot(r, v, c=entry['Color'], alpha=a)
    
    # Plot semi-low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['SemiLow'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,1].plot(r, v, c=entry['Color'], alpha=a)
        
    # Plot semi-high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['SemiHigh'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,0].plot(r, v, c=entry['Color'], alpha=a)
        
    # Plot high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['High'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,1].plot(r, v, c=entry['Color'], alpha=a)

fig_isol_plain

In [ ]:
axes[0,0].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[0,1], isol_bins[0,2] - isol_bins[0,1], 
                   isol_bins[0,1] - isol_bins[0,0]),
               transform=axes[0,0].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[0,1].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[1,1], isol_bins[1,2] - isol_bins[1,1], 
                   isol_bins[1,1] - isol_bins[1,0]),
               transform=axes[0,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[1,0].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[2,1], isol_bins[2,2] - isol_bins[2,1], 
                   isol_bins[2,1] - isol_bins[2,0]),
               transform=axes[1,0].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[1,1].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[3,1], isol_bins[3,2] - isol_bins[3,1], 
                   isol_bins[3,1] - isol_bins[3,0]),
               transform=axes[1,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)

fig_isol_plain

### Save the Figure

In [ ]:
filename = 'isol_rotcurves.png'
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig_isol_plain.savefig(filename, dpi=300, bbox_inches='tight')

---

## Plot Logarithmic Rotation Curves

### Satellite Galaxies

In [ ]:
# Choose font sizes:
parameters = {'font.size': 12,
              'figure.titlesize': 14,
              'axes.labelsize': 12,
              'xtick.labelsize': 10,
              'ytick.labelsize': 10,
              'legend.fontsize': 12}

# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

In [ ]:
# Store the figure into a variable:
fig_sat_log, axes = plt.subplots(nrows=2, ncols=2, sharex='row', sharey='row', figsize=(6, 6))

plt.subplots_adjust(wspace=0.08)
plt.subplots_adjust(hspace=0.2)

# Set logarithmic axes:
axes[0,0].set_xscale('log')
axes[0,0].set_yscale('log')
axes[0,1].set_xscale('log')
axes[0,1].set_yscale('log')
axes[1,0].set_xscale('log')
axes[1,0].set_yscale('log')
axes[1,1].set_xscale('log')
axes[1,1].set_yscale('log')

# Set axis limits:
axes[0,0].set_xlim(0.3, 10)
axes[0,0].set_ylim(6, 40)
axes[1,0].set_xlim(0.3, 30)
axes[1,0].set_ylim(6, 90)

# Set labels:
s=14
axes[1,0].set_xlabel('$r[\mathrm{kpc}]$')
axes[1,1].set_xlabel('$r[\mathrm{kpc}]$')
axes[0,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$')
axes[1,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$')

fig_sat_log.suptitle('Satellite Galaxies', y=0.95)

In [ ]:
# Plot rotation curves:
for i, (name, entry) in enumerate(data.items()):
    
    # Plot low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['Low'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,0].plot(r, v, c=entry['Color'], alpha=a)
    
    # Plot semi-low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['SemiLow'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,1].plot(r, v, c=entry['Color'], alpha=a)
        
    # Plot semi-high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['SemiHigh'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,0].plot(r, v, c=entry['Color'], alpha=a)
        
    # Plot high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Satellite']['High'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Satellite']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,1].plot(r, v, c=entry['Color'], alpha=a)

fig_sat_log

Add bin ranges to the figure explicitly:

In [ ]:
axes[0,0].text(0.08, 0.92,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[0,1], sat_bins[0,2] - sat_bins[0,1], 
                   sat_bins[0,1] - sat_bins[0,0]),
               transform=axes[0,0].transAxes, 
               horizontalalignment='left',
               verticalalignment='top')
axes[0,1].text(0.92, 0.92,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[1,1], sat_bins[1,2] - sat_bins[1,1], 
                   sat_bins[1,1] - sat_bins[1,0]),
               transform=axes[0,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='top')
axes[1,0].text(0.08, 0.92,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[2,1], sat_bins[2,2] - sat_bins[2,1], 
                   sat_bins[2,1] - sat_bins[2,0]),
               transform=axes[1,0].transAxes,
               horizontalalignment='left',
               verticalalignment='top')
axes[1,1].text(0.92, 0.08,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               sat_bins[3,1], sat_bins[3,2] - sat_bins[3,1], 
                   sat_bins[3,1] - sat_bins[3,0]),
               transform=axes[1,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom')

fig_sat_log

### Save the Figure

In [ ]:
filename = 'sat_rotcurves_log.png'
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig_sat_log.savefig(filename, dpi=300, bbox_inches='tight')

### Isolated Galaxies

In [ ]:
# Choose font sizes:
parameters = {'font.size': 12,
              'figure.titlesize': 14,
              'axes.labelsize': 12,
              'xtick.labelsize': 10,
              'ytick.labelsize': 10,
              'legend.fontsize': 12}

# Set fonts:
plt.rcParams.update(parameters)
plt.tight_layout()

In [ ]:
# Store the figure into a variable:
fig_isol_log, axes = plt.subplots(nrows=2, ncols=2, sharex='row', sharey='row', figsize=(6, 6))

plt.subplots_adjust(wspace=0.08)
plt.subplots_adjust(hspace=0.2)

# Set logarithmic axes:
axes[0,0].set_xscale('log')
axes[0,0].set_yscale('log')
axes[0,1].set_xscale('log')
axes[0,1].set_yscale('log')
axes[1,0].set_xscale('log')
axes[1,0].set_yscale('log')
axes[1,1].set_xscale('log')
axes[1,1].set_yscale('log')

# Set axis limits:
axes[0,0].set_xlim(0.3,10)
axes[0,0].set_ylim(6,40)
axes[1,0].set_xlim(0.3,30)
axes[1,0].set_ylim(6,90)

# Set labels:
axes[1,0].set_xlabel('$r[\mathrm{kpc}]$')
axes[1,1].set_xlabel('$r[\mathrm{kpc}]$')
axes[0,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$')
axes[1,0].set_ylabel('$v_{\mathrm{circ}}[\mathrm{kms^{-1}}]$')

fig_isol_log.suptitle('Isolated Galaxies', y=0.95)

In [ ]:
# Plot rotation curves:
for i, (name, entry) in enumerate(data.items()):
    
    # Plot low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['Low'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,0].plot(r, v, c=entry['Color'], alpha=a)
    
    # Plot semi-low mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['SemiLow'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[0,1].plot(r, v, c=entry['Color'], alpha=a)
        
    # Plot semi-high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['SemiHigh'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,0].plot(r, v, c=entry['Color'], alpha=a)
        
    # Plot high mass:
    mask = np.logical_and.reduce([entry['Selections']['VmaxBins']['Isolated']['High'],
                                  entry['Selections']['Luminous'],
                                  entry['Selections']['Isolated']])
    v_circ = entry['Vcirc'][mask]
    radius = entry['Radius'][mask]
    for v, r in zip(v_circ, radius):
        axes[1,1].plot(r, v, c=entry['Color'], alpha=a)

fig_isol_log

Add bin ranges to the figure explicitly:

In [ ]:
axes[0,0].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[0,1], isol_bins[0,2] - isol_bins[0,1], 
                   isol_bins[0,1] - isol_bins[0,0]),
               transform=axes[0,0].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[0,1].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[1,1], isol_bins[1,2] - isol_bins[1,1], 
                   isol_bins[1,1] - isol_bins[1,0]),
               transform=axes[0,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[1,0].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[2,1], isol_bins[2,2] - isol_bins[2,1], 
                   isol_bins[2,1] - isol_bins[2,0]),
               transform=axes[1,0].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)
axes[1,1].text(0.9, 0.1,
               "$v_\mathrm{{max}} = {:.2f}^{{+{:.2f}}}_{{-{:.2f}}} ~\mathrm{{km/s}}$".format(
               isol_bins[3,1], isol_bins[3,2] - isol_bins[3,1], 
                   isol_bins[3,1] - isol_bins[3,0]),
               transform=axes[1,1].transAxes, 
               horizontalalignment='right',
               verticalalignment='bottom',
               fontsize=s)

fig_isol_log

### Save the Figure

In [ ]:
filename = 'isol_rotcurves_log.png'
    
path = os.path.abspath(os.path.join('..', 'Figures', 'MediumResolution'))
filename = os.path.join(path, filename)

fig_isol_log.savefig(filename, dpi=300, bbox_inches='tight')